# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

## Preliminaries: Loading the data and ML model

In [3]:
from dice_ml.utils import helpers # helper functions
dataset = helpers.load_adult_income_dataset().sample(5000) # downsampling to reduce ML model fitting time
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [5]:
import dice_ml
from dice_ml import Dice

d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [6]:
exp = Dice(d, m, method="random")
query_instance = x_train[2:3]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10)
e1.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,42,Private,Some-college,Divorced,White-Collar,White,Female,45,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,85.0,Self-Employed,Prof-school,Separated,Professional,-,Male,89.0,1
1,57.0,Other/Unknown,Masters,Widowed,Sales,-,Male,67.0,1
2,79.0,Self-Employed,Doctorate,Separated,Professional,-,Male,76.0,1
3,-,Self-Employed,Prof-school,Separated,Professional,-,Male,58.0,1
4,17.0,-,Bachelors,Married,-,Other,-,85.0,1
5,66.0,Other/Unknown,Masters,Widowed,Sales,-,Male,56.0,1
6,-,Self-Employed,Prof-school,Separated,Professional,-,Male,26.0,1
7,78.0,Self-Employed,Doctorate,Separated,Sales,-,Male,87.0,1
8,48.0,Self-Employed,Prof-school,Separated,Professional,-,Male,26.0,1
9,56.0,Self-Employed,Doctorate,Widowed,Sales,-,Male,86.0,1


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.local_feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'education': 1.0, 'marital_status': 1.0, 'hours_per_week': 1.0, 'workclass': 0.9, 'occupation': 0.9, 'gender': 0.9, 'age': 0.8, 'race': 0.1}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

[{'education': 1.0, 'marital_status': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'workclass': 0.9, 'occupation': 0.9, 'gender': 0.9, 'race': 0.1}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [9]:
cobj=exp.global_feature_importance(x_train[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

{'hours_per_week': 1.0, 'age': 0.99, 'marital_status': 0.98, 'education': 0.95, 'occupation': 0.87, 'workclass': 0.72, 'gender': 0.56, 'race': 0.44}


## Convert the counterfactual output to json

In [10]:
json_str = cobj.to_json()
print(json_str)

{
  "cf_examples_list": [
    "{\"data_interface\": {\"outcome_name\": \"income\", \"data_df\": \"dummy_data\"}, \"model_type\": \"classifier\", \"desired_class\": \"opposite\", \"desired_range\": null, \"test_instance_df\": \"{\\\"age\\\":{\\\"0\\\":59},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\"},\\\"education\\\":{\\\"0\\\":\\\"Masters\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"White-Collar\\\"},\\\"race\\\":{\\\"0\\\":\\\"Other\\\"},\\\"gender\\\":{\\\"0\\\":\\\"Male\\\"},\\\"hours_per_week\\\":{\\\"0\\\":40},\\\"income\\\":{\\\"0\\\":1}}\", \"final_cfs_df\": \"{\\\"age\\\":{\\\"0\\\":55.0,\\\"1\\\":49.0,\\\"2\\\":41.0,\\\"3\\\":38.0,\\\"4\\\":23.0,\\\"5\\\":35.0,\\\"6\\\":51.0,\\\"7\\\":82.0,\\\"8\\\":17.0,\\\"9\\\":82.0},\\\"workclass\\\":{\\\"0\\\":\\\"Self-Employed\\\",\\\"1\\\":\\\"Government\\\",\\\"2\\\":\\\"Private\\\",\\\"3\\\":\\\"Self-Employed\\\",\\\"4\\\":\\\"Government\\\",\\\"5\\\":\\\"Private\\\",\\\"6\\\":\\\"Other\\\

## Convert the json output to a counterfactual object

In [11]:
imp_r=imp.from_json(json_str)
print([o.visualize_as_dataframe(show_only_changes=True) for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,59,Private,Masters,Married,White-Collar,Other,Male,40,1



Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,55,Self-Employed,Doctorate,Separated,Sales,White,-,4,0
1,49,Government,HS-grad,Single,Service,-,Female,30,0
2,41,-,Assoc,Divorced,Blue-Collar,-,Female,32,0
3,38,Self-Employed,Doctorate,Separated,Sales,White,-,15,0
4,23,Government,HS-grad,Single,Service,-,Female,37,0
5,35,-,Assoc,Divorced,Blue-Collar,-,Female,21,0
6,51,Other/Unknown,-,Widowed,Sales,White,-,20,0
7,82,-,Assoc,Divorced,Blue-Collar,-,Female,14,0
8,17,-,Assoc,Divorced,Blue-Collar,-,Female,22,0
9,82,Government,HS-grad,Single,Service,-,Female,38,0


Query instance (original outcome : 0)


...

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,46,Self-Employed,HS-grad,Married,Professional,White,Male,60,0



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,85,-,Prof-school,Separated,-,-,-,89,1
1,57,Other/Unknown,Masters,Widowed,Sales,-,-,67,1
2,79,-,Doctorate,Separated,-,-,-,76,1
3,45,-,Prof-school,Separated,-,-,-,58,1
4,17,Private,Bachelors,-,White-Collar,Other,Female,85,1
5,66,Other/Unknown,Masters,Widowed,Sales,-,-,56,1
6,39,-,Prof-school,Separated,-,-,-,26,1
7,78,-,Doctorate,Separated,Sales,-,-,87,1
8,48,-,Prof-school,Separated,-,-,-,26,1
9,56,-,Doctorate,Widowed,Sales,-,-,86,1


[None, None, None, None, None, None, None, None, None, None]
[{'marital_status': 1.0, 'occupation': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'education': 0.9, 'gender': 0.7, 'workclass': 0.6, 'race': 0.3}, {'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'race': 0.9, 'gender': 0.9, 'workclass': 0.8}, {'education': 1.0, 'marital_status': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'workclass': 0.9, 'occupation': 0.9, 'gender': 0.9, 'race': 0.1}, {'marital_status': 1.0, 'occupation': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'workclass': 0.9, 'education': 0.8, 'race': 0.7, 'gender': 0.7}, {'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'workclass': 0.9, 'race': 0.1, 'gender': 0.1}, {'marital_status': 1.0, 'occupation': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'education': 0.8, 'workclass': 0.7, 'race': 0.7, 'gender': 0.7}, {'education': 1.0, 'marital_status': 1.0, 'age': 1.0, 'hours_per_week': 1.0, 'oc